In [3]:
import asyncio
import sounddevice as sd
import numpy as np
from scipy.io.wavfile import write
import base64
from pathlib import Path
import websockets
import json

# Duration in seconds
duration = 10
# Sample rate
samplerate = 44100
# Recording audio
print("Recording for 10 seconds")
myrecording = sd.rec(int(samplerate * duration), samplerate=samplerate,
                     channels=2, blocking=True)
print("Recording complete. Saving the audio as output.wav")
write('output.wav', samplerate, myrecording)

def encode_data(filepath: Path) -> str:
    with Path(filepath).open('rb') as fp:
        bytes_data = base64.b64encode(fp.read())
        encoded_data = bytes_data.decode("utf-8")
        return encoded_data

# Encode the .wav file
encoded_data = encode_data('output.wav')

async def send_audio_to_hume(encoded_audio):
    uri = "wss://api.hume.ai/v0/stream/models"
    headers = {"X-Hume-Api-Key": "1Fuo6eVLpIj6ndhmC5VXllArH67eOcaSA0XLX3sHdU2SdEy5"}
    # JSON message to send
    message = {
        "models": {
            "audio": {}
        },
        "raw_audio": True,
        "data": encoded_audio
    }

    async with websockets.connect(uri, extra_headers=headers) as ws:
        await ws.send(json.dumps(message))
        print("Sent message to Hume, waiting for response...")
        response = await ws.recv()
        print("Received response:")
        print(response)

# Run the async function
asyncio.run(send_audio_to_hume(encoded_data))


Recording for 10 seconds
Recording complete. Saving the audio as output.wav


RuntimeError: asyncio.run() cannot be called from a running event loop